Install the prerequisites:

In [1]:
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Login into HuggingFace, so the program can access the stored models. The Llama series requires additional registration with Meta:

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Do the imports:

In [3]:
import torch
from transformers import AutoTokenizer, pipeline

Define the model identifier:

In [4]:
#model = "meta-llama/Llama-2-7b-chat-hf"
model = "openlm-research/open_llama_7b"

Load the tokenizer:

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='openlm-research/open_llama_7b', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


Instantiate text generation pipeline:

In [7]:
text_gen_pipeline = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
print(text_gen_pipeline.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [9]:
sequences = text_gen_pipeline(
    text_inputs='What is the answer to the ultimate question of life, the universe, and everything?',
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=2,
    early_stopping=True
)
for s in sequences:
    print(f"RESULT: {s['generated_text']}")


RESULT: What is the answer to the ultimate question of life, the universe, and everything?
The answer is 42.
The Hitchhiker’s Guide to the Galaxy is a comedy science fiction series created by Douglas Adams. The series is based on the 1978 radio series of the same name. The series was first broadcast on BBC Radio 4 in March 1978. The series was later adapted into a television series, a comic series, and a feature film.
The Hitchhiker’s Guide to the Galaxy is a comedy science
RESULT: What is the answer to the ultimate question of life, the universe, and everything?
The answer is 42.
The Hitchhiker’s Guide to the Galaxy is a comedy science fiction series created by Douglas Adams. The series is based on the 1978 radio series of the same name. The series was first broadcast on BBC Radio 4 in March 1978. The series was later adapted into a television series, a comic series, and a feature film.
The Hitchhiker’s Guide to the Galaxy is set in the


In [10]:
print(sequences)

[{'generated_text': 'What is the answer to the ultimate question of life, the universe, and everything?\nThe answer is 42.\nThe Hitchhiker’s Guide to the Galaxy is a comedy science fiction series created by Douglas Adams. The series is based on the 1978 radio series of the same name. The series was first broadcast on BBC Radio 4 in March 1978. The series was later adapted into a television series, a comic series, and a feature film.\nThe Hitchhiker’s Guide to the Galaxy is a comedy science'}, {'generated_text': 'What is the answer to the ultimate question of life, the universe, and everything?\nThe answer is 42.\nThe Hitchhiker’s Guide to the Galaxy is a comedy science fiction series created by Douglas Adams. The series is based on the 1978 radio series of the same name. The series was first broadcast on BBC Radio 4 in March 1978. The series was later adapted into a television series, a comic series, and a feature film.\nThe Hitchhiker’s Guide to the Galaxy is set in the'}]


In [11]:
sequences = text_gen_pipeline(
    text_inputs='What locations does Pace University Have?',
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=2,
    early_stopping=True
)
for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: What locations does Pace University Have?
Pace University has 19 locations in 4 countries
What is the Pace University campus size?
Pace University campus covers an area of 1,000 acres
What is the Pace University student population?
Pace University student body counts 13,000 students (fall 2020)
What is the Pace University student:teacher ratio?
Pace University has a student population of 13,00
RESULT: What locations does Pace University Have?
Pace University has 19 locations in 4 countries
What is the Pace University campus size?
Pace University campus covers an area of 1,000 acres
What is the Pace University student population?
Pace University student body counts 13,000 students (fall 2020)
What is the Pace University student to faculty ratio?
Pace University has a student to faculty ratio of 14 to 


In [13]:
sequences = text_gen_pipeline(
    text_inputs="Are hamsters good pets?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: Are hamsters good pets?
Hamsters are good pets for children and adults alike. They are very social animals and enjoy being around people. They are also very easy to care for and do not require a lot of space.
What is the lifespan of a hamster?
The lifespan of a hamster can vary depending on the breed and the care that it receives. Generally, hamsters can live for up to 12 years. However, some breeds can live for up to 20


In [14]:
sequences = text_gen_pipeline(
    text_inputs="What is the difference between hamsters and guinea pigs?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: What is the difference between hamsters and guinea pigs?
What is the difference between hamsters and guinea pigs?
Hamsters and guinea pigs are both rodents, but they are not the same. Hamsters are smaller than guinea pigs, and they have a shorter tail. Hamsters also have a different diet than guinea pigs.
What is the difference between a hamster and a guinea pig?
Hamsters and guinea pigs are both rodents, but they are not the same. Hamsters are smaller


In [15]:
sequences = text_gen_pipeline(
    text_inputs="What do hamsters eat?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: What do hamsters eat?
Hamsters are omnivores, which means they eat both plants and animals. Their diet consists mainly of seeds, nuts, fruits, vegetables, and grains. They also eat small insects, such as crickets and mealworms. Hamsters are also known to eat small amounts of meat, such as chicken or beef.
What do hamsters eat in the wild?
In the wild, hamsters eat a variety of foods, including seeds, nuts, fruits, vegetables,


In [16]:
sequences = text_gen_pipeline(
    text_inputs="Are hamsters nocturnal?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: Are hamsters nocturnal?
Hamsters are nocturnal, which means they are most active at night. They are also crepuscular, which means they are most active at dawn and dusk.
What is the lifespan of a hamster?
The average lifespan of a hamster is 1-2 years.
What is the average litter size of a hamster?
The average litter size of a hamster is 4-6 pups.
What is the average


In [17]:
sequences = text_gen_pipeline(
    text_inputs="What other pets are safe to have around hamsters?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: What other pets are safe to have around hamsters?
Hamsters are not safe to have around other pets. They can be aggressive and territorial, and they can also carry diseases that can be passed on to other animals.
What are some good foods for hamsters?
There are many different types of foods that hamsters can eat. Some good foods include:
-Hamster pellets
-Hamster chow
-Hamster treats
-Hamster hay
-Hamster hay cubes
-


In [18]:
sequences = text_gen_pipeline(
    text_inputs="How large should a hamster cage be?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: How large should a hamster cage be?
How big should a hamster cage be?
Hamsters are small rodents that are often kept as pets. They are known for their small size and playful nature, and they can be kept in a variety of cages. One of the most important considerations when choosing a hamster cage is the size of the cage.
The size of the cage should be based on the size of the hamster. Hamsters come in a variety of sizes, and the cage should be large enough


In [19]:
sequences = text_gen_pipeline(
    text_inputs="What toys should I get my hamsters?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: What toys should I get my hamsters?
What is the best food for hamsters?
What is the best hamster cage?
What is the best hamster cage?
What is the best hamster cage?
What is the best hamster cage?
What is the best hamster cage?
What is the best hamster cage?


In [20]:
sequences = text_gen_pipeline(
    text_inputs="Why do people find hamsters to be cuter than other rodents, like mice or rats?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


RESULT: Why do people find hamsters to be cuter than other rodents, like mice or rats?
A: Hamsters are cute because they are small, furry, and have big eyes. They also have a cute little squeak that they make when they are happy.
Q: Why do people find hamsters to be cuter than other rodents, like mice or rats?
A: Hamsters are cute because they are small, furry, and have big eyes. They also have a cute little squeak that they make when they are happy.
Q: Why do


In [21]:
sequences = text_gen_pipeline(
    text_inputs="Why do hamsters like running on wheels?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: Why do hamsters like running on wheels?
Why do hamsters like running on wheels?
Hamsters are small rodents that are native to the Middle East and Asia. They are known for their long tails and their ability to run on wheels. Hamsters are also known for their love of running on wheels.
There are many reasons why hamsters like to run on wheels. One reason is that it helps them to stay fit and healthy. Hamsters need to exercise regularly in order to stay healthy and fit. Running on


In [22]:
sequences = text_gen_pipeline(
    text_inputs="Can you keep hamsters and mice together as pets?",
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=1,
    early_stopping=True
)

for s in sequences:
    print(f"RESULT: {s['generated_text']}")

RESULT: Can you keep hamsters and mice together as pets?
Can you keep hamsters and mice together as pets?
Hamsters and mice are two of the most popular pets in the world. They are both small, cute, and easy to care for. But can you keep hamsters and mice together as pets?
The answer is yes, you can keep hamsters and mice together as pets. However, there are some things you need to know before you do.
Hamsters and mice are both rodents, and they share many
